# Assessing the Economic Impact of Wildfire focus on Building Losses in Texas
I-GUIDE Summer School 2024 Team 5 (Part1)

**Authors**: 
- Zhenlei Song songzl@tamu.edu
- Yoonjung Ahn yjahn@ku.edu
- Yan Xie yanxieyx@umich.edu
- Xiaoyu Xu xiaoyu.xu@siu.edu
- Jinyi Cai jinycai@uiowa.edu
- Sobia Shah sobiashah.libra15@gmail.com
- Harman Singh hxs5376@psu.edu

<br>**Created**: 8-8-24

This notebook introduced an example of assessing the economic impacts of wildfire on building losses in Texas, using multiple machine learning models. The results showed that the gradient boost performs the best with a MSE of 0.185; R2 score of 0.847.

***Please run this notebook using the I-GUIDE JupyterHub `geoai` kernal.***

<a id='top'></a>
## Contents
(Hyperlinks work only within the Jupuyter environment)

***Note:*** `Preprocessing` section takes hours. So we split into 2 notebooks. This is the 1st one, and it stops after preprocessing and save everything into a `.geojson` file. And the 2nd part will load the processed file and move forward on the rest sections.
- [Introduction](#intro)
- [Research Question](#research-question)
- [Data Sources](#data-sources)
- [Methodology](#methods)
    - [Preprocessing](#preprocess)
    - Data Analysis
    - Preparation
    - Machine Learning Models
        - Linear Regression
        - Random Forest
        - Gradient Boosting
- Results
- Discussion
    - Limitations
    - GeoEthics

<a id='intro'></a>
## Introduction

* Wildfire trends globally and in the United States </p>
As a natural phenomenon, wildfires pose substantial economic consequences, and their management is fraught with complexities, dynamic challenges, and incentive problems. 
Fire can directly impact people by posing an immediate threat to life, property, and local economies (Milne et al., 2014). Beyond these direct effects, fire can influence behavior as people evacuate or try to avoid smoke exposure (McCaffrey et al., 2018). Wildfires damage about 
3.3 billion dollars annually in the United States.
In 2020, wildfire damages totaled US 16.5 billion dollars in the United States, with over 10,000 structures damaged or destroyed in California alone (NOAA 2021).

* Why Texas?</p>
Understanding the economic impact of wildfires is crucial, particularly in regions like Texas, where the frequency and intensity of wildfires have been increasing. A prolonged summer in Texas, resulting in repetitive droughts, affects more than 80% of the state and contributes to wildfires (Vu, 2023). According to the U.S. Drought Monitor, nearly 40% of Texas is experiencing extreme or exceptional drought in 2024. Climate change is expected to increase drought severity, further exacerbating wildfire risk. Texas has an average loss of 2.0-5.0 billion dollars annually due to wildfires in 2020.
Moreover, 5683 structures were destroyed, and 372 were destroyed between 2005 and 2003 (Headwaters Economics, 2024). Recent wildfires in Texas have caused extensive property damage, leading to substantial economic losses for local communities—wildfire. The  Bastrop County, Texas Complex fire in 2011 destroyed over 1,600 homes and caused an estimated 325 million dollars in property damage ((Economic Development Corporation). Such events underscore the importance of developing effective wildfire management strategies that consider the economic implications.
 
* Wildfire impact varies according to sociodemograph</p>
Studies have found that the elderly population is among the most vulnerable. Individuals aged 65 and older make up around 12% to 14% of the total population in Texas. The number of residents aged 65 and 4,102 retirees moving to Texas in 2024 indicates that the older population in Texas is expected to grow more

* Purpose of this study</p>
This study aims to synthesize existing economic research on wildfires better to understand their scale, trends, and management phases. By doing so, we aim to highlight the economic interdependencies and spillover effects that create inefficiencies and justify public institutional responses.

* Contribution of this research </p>
While many studies focus on agricultural economic loss, fewer studies examine the economic impact of property loss. This study utilized innovative datasets that describe more detailed environmental factors prone to wildfire (e.g., limited studies have considered climate variables and environmental factors such as fuel disturbance in their analysis, with most studies utilizing vegetation or land use data). Moreover, this study employed multiple machine learning methods to understand the uncertainty and discrepancies among models.




<a id='research-question'></a>
## Research Question

- What is the spatial distribution of EALB?
- How can we more accurately predict the EALB base on environment and climate features? 

<a id='data-sources'></a>
## Data Sources

#### Imports & Configs

In [1]:
!pip install geopandas
import geopandas as gpd
import pandas as pd
import numpy as np
import os

import rasterio 
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask

Defaulting to user installation because normal site-packages is not writeable


In [2]:
BASE_DIR = os.getcwd()
print(f"BASE_DIR is: {BASE_DIR}")
DATASET_DIR = f"/share/Summer-School-2024/Team5/datasets"
print(f"DATASET_DIR is: {DATASET_DIR}")
REPROJECT_PATH = f"{DATASET_DIR}/TX_reprojected"
print(f"REPROJECT_PATH is: {REPROJECT_PATH}")
OUTPUT_PATH = f"{BASE_DIR}/outputs"

BASE_DIR is: /home/jovyan/iguide-szl
DATASET_DIR is: /share/Summer-School-2024/Team5/datasets
REPROJECT_PATH is: /share/Summer-School-2024/Team5/datasets/TX_reprojected


In [3]:
nri_columns = [
        'TRACTFIPS',
        'POPULATION',
        'BUILDVALUE',
        'AGRIVALUE',
        'WFIR_AFREQ',
        'WFIR_EXPB',
        'WFIR_EXPP',
        'WFIR_EXPPE',
        'WFIR_EXPA',
        'WFIR_EXPT',
        'WFIR_HLRP',
        'WFIR_HLRA',
        'WFIR_EALB',
    ]

feature_dict_list = [
    {
        "file_name": f"{DATASET_DIR}/ClimateEngineTifs/DAYMET_Precipitation_JJA.tif",
        "feature_name": "Precipitation",
        "reprojected_file_name": f"{REPROJECT_PATH}/Precipitation.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/ClimateEngineTifs/DAYMET_tmax_JJA.tif",
        "feature_name": "Max-Temperature",
        "reprojected_file_name": f"{REPROJECT_PATH}/MaxTemperature.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/ClimateEngineTifs/DAYMET_tmin_JJA.tif",
        "feature_name": "Min-Temperature",
        "reprojected_file_name": f"{REPROJECT_PATH}/MinTemperature.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/ClimateEngineTifs/DAYMET_tmean_JJA.tif",
        "feature_name": "Mean-Temperature",
        "reprojected_file_name": f"{REPROJECT_PATH}/MeanTemperature.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/ClimateEngineTifs/DAYMET_HargreavesPotentialEvapotranspiration_JJA.tif",
        "feature_name": "Potential-Evapotranspiration",
        "reprojected_file_name": f"{REPROJECT_PATH}/PotentialEvapotranspiration.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX2/BuildingCount_TX.tif",
        "feature_name": "Building-Count",
        "reprojected_file_name": f"{REPROJECT_PATH}/BuildingCount.tif",
        "method": "sum"
    },
    {
        "file_name": f"{DATASET_DIR}/TX2/BuildingCover_TX.tif",
        "feature_name": "Building-Cover",
        "reprojected_file_name": f"{REPROJECT_PATH}/BuildingCover.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX2/BuildingDensity_TX.tif",
        "feature_name": "Building-Density",
        "reprojected_file_name": f"{REPROJECT_PATH}/BuildingDensity.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX2/HUCount_TX.tif",
        "feature_name": "Housing-Unit-Count",
        "reprojected_file_name": f"{REPROJECT_PATH}/HUCount.tif",
        "method": "sum"
    },
    {
        "file_name": f"{DATASET_DIR}/TX2/HUDen_TX.tif",
        "feature_name": "Housing-Unit-Density",
        "reprojected_file_name": f"{REPROJECT_PATH}/HUDen.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX2/HUExposure_TX.tif",
        "feature_name": "Housing-Unit-Exposure",
        "reprojected_file_name": f"{REPROJECT_PATH}/HUExposure.tif",
        "method": "sum"
    },
    
    {
        "file_name": f"{DATASET_DIR}/TX2/PopDen_TX.tif",
        "feature_name": "Population-Density",
        "reprojected_file_name": f"{REPROJECT_PATH}/PopDen.tif",
        "method": "mean"
    },

    {
        "file_name": f"{DATASET_DIR}/TX/CFL_TX.tif",
        "feature_name": "Conditional-Flame-Length",
        "reprojected_file_name": f"{REPROJECT_PATH}/CFL.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX/CRPS_TX.tif",
        "feature_name": "Conditional-Risk-2-Potential-Structures",
        "reprojected_file_name": f"{REPROJECT_PATH}/CRPS.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX/FLEP4_TX.tif",
        "feature_name": "Flame-Length-Over-4-Feet",
        "reprojected_file_name": f"{REPROJECT_PATH}/FLEP4.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX/FLEP8_TX.tif",
        "feature_name": "Flame-Length-Over-8-Feet",
        "reprojected_file_name": f"{REPROJECT_PATH}/FLEP8.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX/RPS_TX.tif",
        "feature_name": "Risk-2-Potential-Structures",
        "reprojected_file_name": f"{REPROJECT_PATH}/RPS.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX/BP_TX.tif",
        "feature_name": "Burn-Probability",
        "reprojected_file_name": f"{REPROJECT_PATH}/BP.tif",
        "method": "mean"
    },
    {
        "file_name": f"{DATASET_DIR}/TX/Exposure_TX.tif",
        "feature_name": "Exposure-type",
        "reprojected_file_name": f"{REPROJECT_PATH}/Exposure.tif",
        "method": "mean"
    },
]

### Census Tract

- Source: [https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::env-ciatool-phase1/explore?layer=4](https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::env-ciatool-phase1/explore?layer=4)
- Downloads: [https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::env-ciatool-phase1/explore?layer=4](https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::env-ciatool-phase1/explore?layer=4)

In [4]:
census_tracts_gdf = gpd.read_file(f"{DATASET_DIR}/tl_2023_48_tract/tl_2023_48_tract.shp")

target_crs = census_tracts_gdf.crs

census_tracts_columns = ['GEOID', 'geometry']
census_tracts_gdf = census_tracts_gdf[census_tracts_columns]
print(census_tracts_gdf.shape)
census_tracts_gdf.head()

(6896, 2)


,GEOID,geometry
0,48157674100,"POLYGON ((-95.61467 29.57828, -95.61339 29.578..."
1,48157674200,"POLYGON ((-95.63989 29.58625, -95.63974 29.586..."
2,48441013501,"POLYGON ((-100.15192 32.08412, -100.15188 32.0..."
3,48441013602,"POLYGON ((-100.14955 32.2816, -100.1495 32.286..."
4,48441013601,"POLYGON ((-100.03974 32.48854, -100.03064 32.4..."


### FEMA NRI

- Source: [https://hazards.fema.gov/nri/wildfire](https://hazards.fema.gov/nri/wildfire)
- Downloads:
  - [https://hazards.fema.gov/nri/data-resources](https://hazards.fema.gov/nri/data-resources)
  - [https://hazards.fema.gov/nri/Content/StaticDocuments/DataDownload//NRI_Table_CensusTracts/NRI_Table_CensusTracts_Texas.zip](https://hazards.fema.gov/nri/Content/StaticDocuments/DataDownload//NRI_Table_CensusTracts/NRI_Table_CensusTracts_Texas.zip)
- Document:
  - [https://www.fema.gov/sites/default/files/documents/fema_national-risk-index_technical-documentation.pdf](https://www.fema.gov/sites/default/files/documents/fema_national-risk-index_technical-documentation.pdf)

In [5]:
nri_df = pd.read_csv(f"{DATASET_DIR}/NRI/NRI_Table_CensusTracts_Texas.csv")

print(nri_df.shape)
nri_df.head()

(6883, 467)


,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,TRACT,...,WNTW_EALS,WNTW_EALR,WNTW_ALRB,WNTW_ALRP,WNTW_ALRA,WNTW_ALR_NPCTL,WNTW_RISKV,WNTW_RISKS,WNTW_RISKR,NRI_VER
0,70112,T48001950100,Texas,TX,48,Anderson,County,1,48001,950100,...,98.913733,Very High,0.000054,2.087558e-07,0.000026,97.593771,106757.149774,99.341198,Very High,March 2023
1,70113,T48001950401,Texas,TX,48,Anderson,County,1,48001,950401,...,95.630270,Relatively High,0.000054,2.087558e-07,0.000026,93.790074,22832.805191,94.124292,Relatively High,March 2023
2,70114,T48001950402,Texas,TX,48,Anderson,County,1,48001,950402,...,97.169833,Relatively High,0.000054,2.087558e-07,0.000026,94.518167,54072.781463,98.072349,Very High,March 2023
3,70115,T48001950500,Texas,TX,48,Anderson,County,1,48001,950500,...,98.085821,Very High,0.000054,2.087558e-07,0.000026,97.054748,79091.415532,98.919041,Very High,March 2023
4,70116,T48001950600,Texas,TX,48,Anderson,County,1,48001,950600,...,98.829180,Very High,0.000054,2.087558e-07,0.000026,97.114639,96776.945351,99.237740,Very High,March 2023


In [6]:

series_nri = nri_df['TRACTFIPS']
series_ct = census_tracts_gdf['GEOID']
series_ct = series_ct.astype('int64')
arr_tmp = np.array(series_ct - series_nri)

### WildFireRisk

- Source: [https://wildfirerisk.org/download/](https://wildfirerisk.org/download/)
- Downloads:
  - [https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0016-2](https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0016-2)
  - [https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0060-2](https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0060-2)
 
This is a raster dataset, dict showing feature layer paths are written in the `feature_dict_list`.

### Climate Engine

- Source: [https://www.climatologylab.org/gridmet.html](https://www.climatologylab.org/gridmet.html)
- Downloads: [https://app.climateengine.org/climateEngine](https://app.climateengine.org/climateEngine)

This is a raster dataset, dict showing feature layer paths are written in the `feature_dict_list`.

### HISDAC-US Building Year

- Source: [https://dataverse.harvard.edu/dataverse/hisdac-usv2](https://dataverse.harvard.edu/dataverse/hisdac-usv2)
- Downloads: [https://dataverse.harvard.edu/dataverse/hisdac-usv2](https://dataverse.harvard.edu/dataverse/hisdac-usv2)
- Description:
    1. **First-year built** (FBUY): values indicate the presence of built structures `built-year`
    2. ***Historical built-up records** (BUPR): Historical built-up property records (BUPR) gridded surfaces

In [7]:
ybhc_df = pd.read_csv(f"{DATASET_DIR}/yearbuilt_housingcount.csv")
ybhc_df = ybhc_df.rename(
    columns={
        "GEOID": "GeoId"
    }
)
ybhc_df.head()

,GeoId,yb_min,yb_max,yb_mean,hc_mean,hc_sum
0,48157674100,1977.0,1994.0,1981.400000,44.980769,2339.0
1,48157674200,1980.0,2010.0,1990.149254,22.246377,1535.0
2,48441013501,1897.0,2020.0,1972.590541,0.207799,2350.0
3,48441013602,1903.0,2020.0,1974.191092,0.262067,2264.0
4,48441013601,1900.0,2018.0,1958.250000,3.974843,1896.0


<a id='methods'></a>
## Methodology

<a id='preprocess'></a>
### Pre-Processing

#### Re-Projection

In [8]:
def reproject_tif(target_crs, source_tif_path, output_tif_path):
    with rasterio.open(source_tif_path) as src:
        transform, width, height = calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        with rasterio.open(output_tif_path, 'w', **kwargs) as dst:
            print(f"Reprojecting {source_tif_path} to {output_tif_path}...")
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest)

In [ ]:
for feature_dict in feature_dict_list:
    reproject_tif(target_crs, feature_dict["file_name"], feature_dict["reprojected_file_name"])

#### Spatial Aggregation

##### Dasymetric mapping

weighted average calculation

- **Calculate Proportionate Area:**
    - For each census tract, calculate the proportion of the tract covered by each class of wildfire factors. This involves overlaying the raster and using spatial analysis tools to determine the area of each factors within each tract.
- **Weighted Average Calculation:**
    - Calculate a weighted average of the wildfire likelihood for each census tract. This can be done by multiplying each likelihood value by the proportion of the census tract it occupies, summing these products, and then dividing by the total area of the census tract:
    Weighted Likelihood=Total Tract Area∑(Likelihood Value×Area Covered)

In [ ]:
# Aggregation
def add_column_by_feature(gdf, feature_dict):
    raster = rasterio.open(feature_dict["reprojected_file_name"])
    print(f"Fetching values for {feature_dict['feature_name']}...")
    value_list = []
    for _, row in gdf.iterrows():
        geom = [row['geometry'].__geo_interface__]
        out_image, out_transform = mask(raster, geom, crop=True)
        masked = np.ma.masked_array(out_image, out_image == raster.nodata)
        if feature_dict["method"] == "sum":
            sum_value = masked.sum()
            # row[feature_name] = sum_value
            value_list.append(sum_value)
        elif feature_dict["method"] == "mean":
            mean_value = masked.mean()
            # row[feature_name] = mean_value
            value_list.append(mean_value)

    raster.close()
    print(f"Adding {feature_dict['feature_name']} to the GeoDataFrame...")
    gdf.insert(2, feature_dict["feature_name"], value_list)

In [ ]:
# Initialize a list to store the results
results = []

for _, tract in census_tracts_gdf.iterrows():
    # Extract the geometry in GeoJSON format
    geom = [tract['geometry'].__geo_interface__]

    results.append({
        'GeoId': tract['GEOID'],  # 'GEOID' is the identifier for the census tract
        'geometry': tract['geometry']  # Add the geometry to the result
    })

gdf = gpd.GeoDataFrame(
    results,
    geometry='geometry',
    crs=census_tracts_gdf.crs  # Use the same CRS as the census tracts
)
gdf['GeoId'] = gdf['GeoId'].astype("int64")
# merge NRI data with census tract data
merged_gdf = gdf.merge(
    nri_df, left_on='GeoId', right_on='TRACTFIPS', how='left')



In [ ]:
merged_gdf.shape

In [ ]:
# clean null rows
merged_gdf = merged_gdf.dropna(subset=nri_columns)

merged_gdf = merged_gdf.merge(
    ybhc_df, left_on='GeoId', right_on='GeoId', how='left'
)

for feature_dict in feature_dict_list:
    add_column_by_feature(merged_gdf, feature_dict)
    # add_column_by_feature(gdf, feature_dict["reprojected_file_name"], feature_dict["feature_name"])

# change columns from string to float

merged_gdf['Min-Temperature'] = merged_gdf['Min-Temperature'].apply(lambda x: np.nan if np.ma.is_masked(x) else x).astype("float64")
merged_gdf['Max-Temperature'] = merged_gdf['Max-Temperature'].apply(lambda x: np.nan if np.ma.is_masked(x) else x).astype("float64")
merged_gdf['Mean-Temperature'] = merged_gdf['Mean-Temperature'].apply(lambda x: np.nan if np.ma.is_masked(x) else x).astype("float64")

merged_gdf = merged_gdf.convert_dtypes()
print(merged_gdf.dtypes)

### Remove the comments if you want to save files.

# merged_gdf.to_file(f"{DATASET_DIR}/results.geojson", driver='GeoJSON')
# merged_df = pd.DataFrame(merged_gdf.drop(columns=['geometry', 'GeoId', 'TRACTFIPS', 'WFIR_HLRB', 'WFIR_HLRR']))
# merged_df.to_csv(f"{DATASET_DIR}/results.csv", index=False)